In [5]:
import asyncio

# The `asyncio` Library

## Coroutine

From the [Fluent Python](https://www.fluentpython.com/extra/classic-coroutines/#intro), there are three kinds of coroutines in Python:

* classic coroutines: A generator function that consumes data sent to it via `coro_func.send(data)` calls, and reads that data by using `yield` in an expression. 

* generator-based coroutines: A generator function decorated with `@types.coroutine`, which makes it compatible with the new `await` keyword, introduced in Python 3.5.

* native coroutines: A coroutine defined with `async def`. We can delegate from a native coroutine to another native coroutine or to a generator-based coroutine using the `await` keyword, similar to how classic coroutines use `yield from`.

```python
#!/usr/bin/env python3

import asyncio

async def async_count() -> None:
    """
    Asynchronously prints 'One', waits for 1 second, then prints 'Two'.
    """
    print('One')
    await asyncio.sleep(1)
    print('Two')
    
async def main() -> None:
    """
    Runs three async_count tasks concurrently.
    """
    await asyncio.gather(
        async_count(), async_count(), async_count()
    )
    
if __name__ == "__main__":
    import time
    import os
    start_time = time.perf_counter()
    asyncio.run(main())
    elapsed = time.perf_counter() - start_time
    file_name = os.path.basename(__file__)
    print(f"The module {file_name} executed in {elapsed:.2f} seconds")
```

In [2]:
!python3 ../scripts/count_async.py

One
One
One
Two
Two
Two
The module count_async.py executed in 1.00 seconds


## Event Loop

The event loop is the core of every `asyncio` application. We can think of it as a conductor that will manage all the asynchronous tasks sent to it. In the example above, the `asyncio.run` starts the event loop and returns only when the event loop exits.

When each task reaches the `await asyncio.sleep(1)` line, the keyword `await` returns the function control to the event loop to run other tasks, suspending the execution of the surrounding coroutine. After 1 second, the task will resume and print 'Two'.

This can be contrasted with the synchronous version of the code:

```python
#!/usr/bin/env python3
import time

def count() -> None:
    """ 
    Synchronous count function.
    """
    print("One")
    time.sleep(1)
    print("Two")
    
def main() -> int:
    for i in range(3):
        count()
    
    return 0

if __name__ == "__main__":
    
    main()
```

In [4]:
!python3 ../scripts/count_sync.py

One
Two
One
Two
One
Two
The module count_sync.py executed in  3.01 seconds


In the synchronous version, the `time.sleep(1)` call will block the entire program for 1 second. The benefit of using `asyncio.sleep` is that it will not block the entire program, allowing other tasks to run while it waits.

## Rules of Async IO

1. When a `await f()` expression is encountered in the scope of a native coroutine, the `await` keyword will pause the execution of the coroutine until the awaited result is available.

In [6]:
async def async_count() -> None:
    print("One")
    # Pause execution here and return to `async_count` when `asyncio.sleep(1)` is done
    await asyncio.sleep(1)
    print("Two")

2. Using `await` or `return` creates a coroutine function. To call a coroutine function, we must `await` it to get its results.

    ```python
    # This is valid
    async def f(x):
        y = await z(x)  # Both `await` and `return` are allowed in coroutines
        return y
    ```

3. Anything defined with `async def` may not use `yield from`, which will raise a `SyntaxError`.

    ```python
    # SyntaxError: 'yield from' inside async function
    async def m(x):
        yield from gen(x)
    ```

4. The keyword `await` can only be used inside a native coroutine. Using it in a non-coroutine function will raise a `SyntaxError`.

    ```python
    # SyntaxError: 'await' outside async function
    def m(x):
        y = await z(x)  
        return y
    ```

5. The `for` keyword works with `iterables`. The `await` keyword works with `awaitables`. As an end user of `asyncio`, these are the awaitables we will see on a daily basis:
    * A native coroutine object, which we get by calling a native coroutine function
    * An `asyncio.Task`, which we usually get by passing a coroutine object to `asyncio.create_task()`

## Probing Domains

This is a program that uses `asyncio` to probe a list of domains concurrently.

```python
#!/usr/bin/env python3
import asyncio
import socket
from keyword import kwlist
from typing import Tuple

# Set maximum length of keyword for domains
MAX_KEYWORD_LEN = 4

async def probe(domain: str) -> Tuple[str, bool]:
    """
    Check domains to see if they are available.

    Parameters
    ----------
    domain : str
        A domain name constructed using a python keyword ending with .dev

    Returns
    -------
    Tuple[str, bool]
        A tuple (domain name, boolean), where True means the domain is resolved and False means that it may be available.
    """
    # Get reference to the event loop
    loop = asyncio.get_running_loop()
    try:
        # This coroutine returns (family, type, proto, canonname, sockaddr), used to connect to the given address via socket
        await loop.getaddrinfo(host=domain, port=None)
    except socket.gaierror:
        # If the domain is not a valid hostname, it may be available
        return (domain, False)
    return (domain, True)

async def main() -> None:
    
    # Generator of keywords and domain names
    names = (key_word for key_word in kwlist if len(key_word) <= MAX_KEYWORD_LEN)
    domains = (f"{name}.dev".lower() for name in names)
    # List of coroutines objects
    coros = [probe(domain=domain) for domain in domains]

    # Yield coroutines, which return the results of the coroutines, in the order they are completed
    for coro in asyncio.as_completed(coros):
        # If `coro` raised an unhandled exception, it would be re-raised here
        domain, found = await coro
        indicator = '+' if found else ' '
        print(f"{indicator} {domain}")

if __name__ == "__main__":
    
    import time
    s = time.perf_counter()
    asyncio.run(main())
    elapsed = time.perf_counter() - s
    print(f"The program executed in {elapsed: .2f} seconds")
```

In [1]:
!python3 ../scripts/blogdom_async.py

+ def.dev
  pass.dev
  with.dev
+ and.dev
  elif.dev
+ from.dev
+ not.dev
  else.dev
  is.dev
  for.dev
+ try.dev
  or.dev
  none.dev
  if.dev
+ as.dev
+ in.dev
  true.dev
+ del.dev
The program executed in  5.96 seconds


In the example above, the `probe` coroutine checks if a domain is available by attempting to resolve it. If the domain is not a valid hostname, it may be available. The `main` coroutine creates a list of coroutine objects and yields them using `asyncio.as_completed`. This function, `asyncio.as_completed`, returns a generator that yields the results of the coroutines in the order they are completed.

The time needed to probe all the domains is the time needed to probe the slowest domain. This is because the `probe` coroutine is an I/O-bound task, and the `asyncio` library will run all the tasks concurrently.

### Non-Blocking

The `await loop.getaddrinfo(...)` does the following:

Basic Operation:

* `await loop.getaddrinfo(...)` suspends the current coroutine (e.g. `probe('if.dev')`) and avoids blocking. This allows the event loop to manage other tasks concurrently.

The process is:

* **Coroutine Creation**: A new coroutine object is created by `getaddrinfo('if.dev', None)`.

* **Awaiting Coroutine**: When `await` is used, it:
    - Starts the low-level addrinfo query.
    - Yields control back to the event loop, allowing it to handle other activities.

* **Event Loop Management**: Other coroutines, like `probe('or.dev')`, can run while `probe('if.dev')` is suspended.

Upon Completion:

* **Response Handling**: Once `getaddrinfo('if.dev', None)` resolves, the event loop gets a response for the `getaddrinfo` query. The event loop:
    - Resumes the suspended coroutine.
    - Passes control back to `probe('if.dev')`.
  
* **Exception and Result Handling**: The resumed coroutine can now:
    - Manage exceptions.
    - Return results.